In [1]:
import pandas as pd
import numpy as np
import pickle
from faker import Faker
import df_helper as dfh

df_helper = dfh.df_helper()

# Initialize Faker
fake = Faker()

admission_types = [
    'URGENT', 'ELECTIVE', 'EW EMER.', 'DIRECT EMER.', 'EU OBSERVATION',
    'OBSERVATION ADMIT', 'DIRECT OBSERVATION',
    'AMBULATORY OBSERVATION', 'SURGICAL SAME DAY ADMISSION'
]
admission_locations = [
    'TRANSFER FROM HOSPITAL', 'TRANSFER FROM SKILLED NURSING FACILITY',
    'INTERNAL TRANSFER TO OR FROM PSYCH', 'PHYSICIAN REFERRAL',
    'EMERGENCY ROOM', 'PACU', 'PROCEDURE SITE',
    'WALK-IN/SELF REFERRAL', 'INFORMATION NOT AVAILABLE',
    'CLINIC REFERRAL'
]
insurance_types = [
    'Medicaid', 'Medicare', 'Other'
]
languages = [
    'ENGLISH', '?'
]
marital_statuses = [
    'SINGLE', 'MARRIED', 'DIVORCED', 'WIDOWED'
]
races = [
    'BLACK/CAPE VERDEAN', 'HISPANIC/LATINO - PUERTO RICAN', 'WHITE',
    'UNKNOWN', 'OTHER', 'BLACK/AFRICAN AMERICAN',
    'HISPANIC/LATINO - SALVADORAN', 'UNABLE TO OBTAIN',
    'WHITE - OTHER EUROPEAN', 'PORTUGUESE', 'HISPANIC/LATINO - CUBAN',
    'PATIENT DECLINED TO ANSWER', 'WHITE - BRAZILIAN',
    'HISPANIC OR LATINO'
]

drugs = [
    'Midodrine', 'Multivitamins W/minerals', 'Sodium Chloride',
    'Nephrocaps', 'Nicotine Polacrilex', 'OLANZapine', 'Ondansetron',
    'OxyCODONE (Immediate Release)'
]

diagnoses_types = [
    'Urinary tract infection, site not specified',
    'Acute respiratory failure',
    'Asthma, unspecified type, unspecified',
    'Solitary pulmonary nodule',
    'Streptococcus infection in conditions classified elsewhere and of unspecified site, streptococcus, group D [Enterococcus]',
    'Dysphagia, oropharyngeal phase', 'Esophageal reflux', 'Hypoxemia',
    'Methicillin resistant pneumonia due to Staphylococcus aureus',
    'Attention deficit disorder with hyperactivity'
]

# List of sample medical supplies and equipment categories
inventory_categories = [
    'Medical Equipment', 'Pharmaceuticals', 'Surgical Tools',
    'Diagnostic Tools', 'PPE', 'Cleaning Supplies'
]

# List of sample item names
inventory_items = {
    'Medical Equipment': ['X-Ray Machine', 'Ultrasound Machine', 'MRI Scanner', 'ECG Monitor'],
    'Pharmaceuticals': ['Aspirin', 'Ibuprofen', 'Amoxicillin', 'Paracetamol'],
    'Surgical Tools': ['Scalpel', 'Forceps', 'Sutures', 'Hemostat'],
    'Diagnostic Tools': ['Thermometer', 'Blood Pressure Monitor', 'Pulse Oximeter'],
    'PPE': ['Face Masks', 'Gloves', 'Gowns', 'Face Shields'],
    'Cleaning Supplies': ['Disinfectant', 'Sanitizer', 'Alcohol Swabs', 'Wipes']
}

# Function to generate a random inventory item
def generate_inventory_item():
    category = np.random.choice(inventory_categories)
    item_name = np.random.choice(inventory_items[category])
    quantity = np.random.randint(10, 500)  # Random quantity of items
    price_per_unit = round(np.random.uniform(5, 5000), 2)  # Random price per unit
    expiry_date = fake.date_between(start_date='today', end_date='+3y')  # Random expiry date within 3 years
    return {
        'Category': category,
        'Item Name': item_name,
        'Quantity': quantity,
        'Price per Unit ($)': price_per_unit,
        'Total Value ($)': round(quantity * price_per_unit, 2),
        'Expiry Date': expiry_date
    }

# Generate hospital inventory data
def generate_hospital_inventory(num_items):
    inventory = []
    for _ in range(num_items):
        inventory.append(generate_inventory_item())
    return pd.DataFrame(inventory)

def generate_patients(n):
    patients = []
    for i in range(n):
        gender = np.random.choice(['M', 'F'])
        patients.append({
            'patient_id': i,
            'name': fake.name_male() if gender == "M" else fake.name_female(),
            'age': np.random.randint(1, 90),
            'gender': gender,
            'insurance': np.random.choice(insurance_types),
            'language': np.random.choice(languages),
            'maritalStatus': np.random.choice(marital_statuses),
            'race': np.random.choice(races),
        })
    return pd.DataFrame(patients)

def generate_staff(n):
    staff = []
    for i in range(n):
        staff.append({
            'staff_id': i,
            'staff_name': fake.name(),
            'role': np.random.choice(['Physician', 'Nurse', 'Admin']),
            'shift_start': fake.time(),
            'shift_end': fake.time()
        })
    return pd.DataFrame(staff)

def generate_admissions(patients:pd.DataFrame):
    admissions = []
    for i in range(len(patients)):
        admissions.append({
            'adm_id': i,
            'patient_id': patients.iloc[i].patient_id,
            'admit_time': fake.date_time(),
            'discharge_time': None,
            'type': np.random.choice(admission_types),
            'location': np.random.choice(admission_locations),
            
        })
    return pd.DataFrame(admissions)

def generate_omr(admissions):
    table = []
    for i in range(len(admissions)):
        omr = {
            'omr_id': i,
            'adm_id': admissions.iloc[i].adm_id,
            'weight': np.random.randint(60, 120),
            ##'height': np.random.randint(150, 220)/ 10.0, #not used in modeling
            'bp_systolic': np.random.randint(80, 220),
            'bp_diastolic': np.random.randint(60, 120),
        }
        table.append(omr)
    return pd.DataFrame(table)

def generate_prescriptions(admissions):
    prescriptions = []
    for i in range(len(admissions)):
        prescriptions.append({
            'prescription_id': i,
            'adm_id': admissions.iloc[i].adm_id,
            'drug': np.random.choice(drugs),
        })
    return pd.DataFrame(prescriptions)

def generate_diagnoses(admissions:pd.DataFrame):
    diag = []
    for i in range(len(admissions)):
        diag.append({
            'diagnosis_id': i,
            'adm_id': admissions.iloc[i].adm_id,
            'diagnosis': np.random.choice(diagnoses_types),
        })
    return pd.DataFrame(diag)

def generate_rooms(n):
    rooms = []
    beds = []
    bed_taken = 0
    room_id = 0
    bed_id = 0
    for i in range(n):
        if i % 2 == 0:#create new room
            rooms.append({
                'room_id': room_id,
                'capacity': 1,
            })
            beds.append({
                "bed_id": bed_id,
                "room": room_id,
                "adm_id": None
            })
            room_id += 1
        else:
            room = np.random.choice(rooms)#add bed to existing room
            room["capacity"] += 1
            beds.append({
                "bed_id": bed_id,
                "room": room["room_id"],
                "adm_id": None
            })
        bed_id += 1

    return pd.DataFrame(rooms), pd.DataFrame(beds)

def assign_beds(admissions, beds):
    for i in range(len(admissions)):
        adm = admissions.iloc[i].adm_id
        bed = beds[beds.adm_id.isna()].sample(1).iloc[0,0]  #choose random empty bed
        beds.loc[bed, "adm_id"] = adm.astype(int)
        print(bed)

In [2]:
hospital_inventory = generate_hospital_inventory(50)
staff = generate_staff(20)
patients = generate_patients(30)
admissions = generate_admissions(patients)
admissions.discharge_time = pd.to_datetime(admissions.discharge_time)
omr = generate_omr(admissions)
prescriptions = generate_prescriptions(admissions)
diagnoses = generate_diagnoses(admissions)
rooms, beds = generate_rooms(50)
assign_beds(admissions, beds)

11
22
4
1
43
26
38
46
13
29
39
15
14
10
16
41
45
5
8
17
36
30
0
19
23
12
21
28
35
20


In [3]:
hospital_inventory.head()

,Category,Item Name,Quantity,Price per Unit ($),Total Value ($),Expiry Date
0,Diagnostic Tools,Blood Pressure Monitor,120,2893.11,347173.20,2025-09-04
1,Cleaning Supplies,Sanitizer,173,3838.98,664143.54,2024-12-10
2,Diagnostic Tools,Thermometer,444,3736.92,1659192.48,2027-07-24
3,Diagnostic Tools,Blood Pressure Monitor,94,2500.72,235067.68,2024-11-30
4,Cleaning Supplies,Sanitizer,341,3902.76,1330841.16,2026-06-16


In [4]:
staff.head()

,staff_id,staff_name,role,shift_start,shift_end
0,0,Jorge Carter,Nurse,16:05:02,13:29:24
1,1,Matthew Moore,Nurse,09:41:08,05:53:09
2,2,Brandon Thomas,Nurse,16:52:50,20:43:57
3,3,Matthew Decker,Physician,01:37:53,08:30:19
4,4,Stephen Bell,Nurse,06:51:58,12:02:20


In [5]:
patients.head()

,patient_id,name,age,gender,insurance,language,maritalStatus,race
0,0,Erin Ross,25,F,Medicare,ENGLISH,WIDOWED,WHITE - BRAZILIAN
1,1,Richard Moore,76,M,Medicaid,ENGLISH,MARRIED,WHITE - OTHER EUROPEAN
2,2,Shannon Carter,50,F,Other,?,DIVORCED,BLACK/AFRICAN AMERICAN
3,3,Jason Keith,56,M,Medicaid,?,SINGLE,WHITE
4,4,Bryan Lee,36,M,Medicare,ENGLISH,WIDOWED,WHITE - BRAZILIAN


In [6]:
admissions

,adm_id,patient_id,admit_time,discharge_time,type,location
0,0,0,1986-03-16 01:50:46.306053,NaT,EW EMER.,PHYSICIAN REFERRAL
1,1,1,1994-09-23 22:33:41.115142,NaT,OBSERVATION ADMIT,CLINIC REFERRAL
2,2,2,2010-01-13 16:46:22.045298,NaT,SURGICAL SAME DAY ADMISSION,TRANSFER FROM SKILLED NURSING FACILITY
3,3,3,2022-01-24 09:41:33.566597,NaT,EW EMER.,EMERGENCY ROOM
4,4,4,2004-04-27 13:34:45.631775,NaT,SURGICAL SAME DAY ADMISSION,TRANSFER FROM SKILLED NURSING FACILITY
5,5,5,1999-04-04 12:49:26.929913,NaT,DIRECT OBSERVATION,PHYSICIAN REFERRAL
6,6,6,2012-10-29 14:00:04.752644,NaT,AMBULATORY OBSERVATION,TRANSFER FROM HOSPITAL
7,7,7,2005-09-30 14:45:16.026422,NaT,ELECTIVE,CLINIC REFERRAL
8,8,8,1992-10-05 04:47:01.430513,NaT,URGENT,PHYSICIAN REFERRAL
9,9,9,1971-03-29 01:00:52.986062,NaT,DIRECT OBSERVATION,TRANSFER FROM HOSPITAL


In [7]:
admissions.iloc[0,0]

0

In [8]:
omr.head()

,omr_id,adm_id,weight,bp_systolic,bp_diastolic
0,0,0,115,210,72
1,1,1,81,141,118
2,2,2,64,165,96
3,3,3,106,183,62
4,4,4,110,151,79


In [9]:
prescriptions.head()

,prescription_id,adm_id,drug
0,0,0,Nicotine Polacrilex
1,1,1,Multivitamins W/minerals
2,2,2,Nicotine Polacrilex
3,3,3,OLANZapine
4,4,4,Midodrine


In [10]:
diagnoses.head()

,diagnosis_id,adm_id,diagnosis
0,0,0,Methicillin resistant pneumonia due to Staphyl...
1,1,1,Acute respiratory failure
2,2,2,Esophageal reflux
3,3,3,"Urinary tract infection, site not specified"
4,4,4,Hypoxemia


In [11]:
rooms

,room_id,capacity
0,0,5
1,1,3
2,2,4
3,3,3
4,4,3
5,5,1
6,6,2
7,7,3
8,8,1
9,9,3


In [12]:
beds

,bed_id,room,adm_id
0,0,0,22
1,1,0,3
2,2,1,None
3,3,0,None
4,4,2,2
5,5,1,17
6,6,3,None
7,7,0,None
8,8,4,18
9,9,2,None


In [13]:
with open('app/data/staff.csv', 'w') as f:
    staff.to_csv(f, index=False)
with open('app/data/patients.csv', 'w') as f:
    patients.to_csv(f, index=False)
with open('app/data/admissions.csv', 'w') as f:
    admissions.to_csv(f, index=False)
with open('app/data/omr.csv', 'w') as f:
    omr.to_csv(f, index=False)
with open('app/data/prescriptions.csv', 'w') as f:
    prescriptions.to_csv(f, index=False)
with open("app/data/diagnoses.csv", "w") as f:
    diagnoses.to_csv(f, index=False)
with open("app/data/rooms.csv", "w") as f:
    rooms.to_csv(f, index=False)
with open("app/data/beds.csv", "w") as f:
    beds.to_csv(f, index=False)

In [14]:
with open("app/data/admission_types.pkl", "wb") as f:
    pickle.dump(admission_types, f)
with open("app/data/admission_locations.pkl", "wb") as f:
    pickle.dump(admission_locations, f)
with open("app/data/insurance_types.pkl", "wb") as f:
    pickle.dump(insurance_types, f)
with open("app/data/languages.pkl", "wb") as f:
    pickle.dump(languages, f)
with open("app/data/marital_statuses.pkl", "wb") as f:
    pickle.dump(marital_statuses, f)
with open("app/data/races.pkl", "wb") as f:
    pickle.dump(races, f)
with open("app/data/drugs.pkl", "wb") as f:
    pickle.dump(drugs, f)
with open("app/data/diagnoses_types.pkl", "wb") as f:
    pickle.dump(diagnoses_types, f)

In [15]:
patient_db = pd.merge(patients, admissions, how="inner", on="patient_id")
patient_db = pd.merge(patient_db, omr, how="inner", on="adm_id")
patient_db = pd.merge(patient_db, prescriptions, how="inner", on="adm_id")
patient_db = pd.merge(patient_db, diagnoses, how="inner", on="adm_id")

df_helper.drop_columns(
    patient_db,[
        'language',
        'admit_time', 
        'discharge_time',
        'omr_id',
        'prescription_id', 
        'diagnosis_id',
])
patient_db

,patient_id,name,age,gender,insurance,maritalStatus,race,adm_id,type,location,weight,bp_systolic,bp_diastolic,drug,diagnosis
0,0,Erin Ross,25,F,Medicare,WIDOWED,WHITE - BRAZILIAN,0,EW EMER.,PHYSICIAN REFERRAL,115,210,72,Nicotine Polacrilex,Methicillin resistant pneumonia due to Staphyl...
1,1,Richard Moore,76,M,Medicaid,MARRIED,WHITE - OTHER EUROPEAN,1,OBSERVATION ADMIT,CLINIC REFERRAL,81,141,118,Multivitamins W/minerals,Acute respiratory failure
2,2,Shannon Carter,50,F,Other,DIVORCED,BLACK/AFRICAN AMERICAN,2,SURGICAL SAME DAY ADMISSION,TRANSFER FROM SKILLED NURSING FACILITY,64,165,96,Nicotine Polacrilex,Esophageal reflux
3,3,Jason Keith,56,M,Medicaid,SINGLE,WHITE,3,EW EMER.,EMERGENCY ROOM,106,183,62,OLANZapine,"Urinary tract infection, site not specified"
4,4,Bryan Lee,36,M,Medicare,WIDOWED,WHITE - BRAZILIAN,4,SURGICAL SAME DAY ADMISSION,TRANSFER FROM SKILLED NURSING FACILITY,110,151,79,Midodrine,Hypoxemia
5,5,Jessica Haas,12,F,Other,SINGLE,PATIENT DECLINED TO ANSWER,5,DIRECT OBSERVATION,PHYSICIAN REFERRAL,81,219,90,Nephrocaps,Solitary pulmonary nodule
6,6,Maria Peterson,57,F,Medicare,WIDOWED,HISPANIC OR LATINO,6,AMBULATORY OBSERVATION,TRANSFER FROM HOSPITAL,70,136,111,OxyCODONE (Immediate Release),Esophageal reflux
7,7,Joshua Harper,11,M,Medicaid,MARRIED,WHITE - OTHER EUROPEAN,7,ELECTIVE,CLINIC REFERRAL,89,105,114,Sodium Chloride,Streptococcus infection in conditions classifi...
8,8,Christopher Hardy,45,M,Medicaid,MARRIED,PORTUGUESE,8,URGENT,PHYSICIAN REFERRAL,98,176,109,Multivitamins W/minerals,Streptococcus infection in conditions classifi...
9,9,Kimberly Burke,47,F,Medicaid,SINGLE,UNABLE TO OBTAIN,9,DIRECT OBSERVATION,TRANSFER FROM HOSPITAL,113,122,96,OLANZapine,"Asthma, unspecified type, unspecified"


In [16]:
with open("app/data/patient_db.csv", "w") as f:
    patient_db.to_csv(f, index=False)
with open("app/data/patient_db.csv", "w") as f:
hospital_inventory.to_csv('hospital_inventory.csv', index=False)